If you don't have a virtual environment setup and want to try things quickly, you might want to uncomment and run:

In [ ]:
# pip install -e ../..

## Generating Environments

Print out the mouselab mdp environment registry to see if the environment we're studying is already there:

In [ ]:
from mouselab.envs.registry import registry

print(registry)

### No matching experiment setting?

If the experiment setting you want to use is not printed out in the registry of the mouselab package, you have two options:
 
1. Add a new experiment setting to the mouselab package registry (use examples from `mouselab/envs/__init__.py`) and submit a pull request.
2. Locally define the experiment setting and add it to the registry -- we will show two examples now.

Say we want to investigate an environment with branching factor 1, 1, 1, 1, 3:
```

        depth  depth   depth   depth    depth
start     1      2       3       4        5
                                       ()
                                     /
                                    /
                                   /
 () ---- () ---- () ----() ----- () ---- ()
                                    \
                                     \
                                      \
                                       ()
 ```
 
You have two reward settings you're interested in:
 
- Increasing variance, that is the same as the `large_increasing` environment above.
- Decreasing variance, that is the opposite direction of the `large_increasing` environment above.

In [ ]:
from mouselab.envs.registry import register
from mouselab.distributions import Normal
from mouselab.envs.reward_settings import large_increasing_reward

# create narrow_large_increasing
register(
    name="narrow_large_increasing",
    branching=[1,1,1,1,3],
    reward_inputs=["depth"],
    reward_dictionary=large_increasing_reward,
)

large_decreasing_reward = {
    level_idx + 1: Normal(0, level) for level_idx, level in enumerate([32, 8, 4, 2, 1])
}


# create narrow_large_decreasing
register(
    name="narrow_large_decreasing",
    branching=[1,1,1,1,3],
    reward_inputs=["depth"],
    reward_dictionary=large_decreasing_reward,
)

You can now check that your environments are on the registry:

In [ ]:
print(registry)

You can get one environment's details using:

In [ ]:
registry("narrow_large_increasing").branching, registry("narrow_large_increasing").reward_function

Now let's instantiate two Mouselab MDP gym environments:

In [ ]:
from mouselab.mouselab import MouselabEnv
from mouselab.distributions import Categorical


env_increasing = MouselabEnv.new_symmetric_registered("narrow_large_increasing",
                                           cost=1)
env_decreasing = MouselabEnv.new_symmetric_registered("narrow_large_decreasing",
                                           cost=1)

For future steps we will keep using `env_increasing`.

Once finished you could print out the current state of the environment:

In [ ]:
print(env_increasing._state)
# Output: ` (0, Norm(0.00, 1.00), Norm(0.00, 2.00), Norm(0.00, 4.00), Norm(0.00, 8.00), Norm(0.00, 32.00), Norm(0.00, 32.00), Norm(0.00, 32.00))`

Now say you see what happens when you reveal the 3rd node:

In [ ]:
env_increasing._step(3)
print(env_increasing._state)
# Output: ` (0, Norm(0.00, 1.00), Norm(0.00, 2.00), <number>, Norm(0.00, 8.00), Norm(0.00, 32.00), Norm(0.00, 32.00), Norm(0.00, 32.00))`

## Exact Solution

The code for the exact solution can be found in exact.py . There are some limitations on size and distributions (we only want to use categorical distributions, but there is a method to discretize implemented continuous distributions.) Since we are planning over belief states, you should only need to run it once for each reward configuration. The `timed_solve_env` function in `exact_utils.py` is probably good enough depending on what you want as output. 

If you want to try the solver out on the environment created above:

_Note: For the 3-1-2 branching environment, this will probably take around 30 minutes to two hours depending on your machine. Therefore, we use a smaller environment in this example._

Now we want to investigate an environment with branching factor 1, 1, 3:
```

        depth  depth   depth    
start     1      2       3           
                          ()
                         /
                       /
                     /
 () ---- () ---- ()  -----()
                     \
                      \
                       \
                        ()
 ```

And which shares the reward function of the `high_increasing` environment.

In [ ]:
from mouselab.envs.reward_settings import high_increasing_reward

register(
    name="narrow_high_increasing",
    branching=[1,1,3],
    reward_inputs=["depth"],
    reward_dictionary=high_increasing_reward,
)

In [ ]:
from mouselab.exact_utils import timed_solve_env


env_increasing = MouselabEnv.new_symmetric_registered("narrow_high_increasing", cost=1)

env_increasing.reset()
q, v, pi, info = timed_solve_env(env_increasing)

The first three outputs are functions (i.e. q(s,a), v(s) and pi(s).)

#### Saving a solution

If you would like to save the Q files, you can save them as a dictionary with keys of (s,a) tuples. Set the `save_q` flag in the `solve` or `timed_solve_env` function to `True`. Please note that solving for the actual Q-values vs solving for the value of an environment can be more costly because we do not (currently) exploit the symmetry between states with a hash function.

In [ ]:
env_increasing.reset()
q, v, pi, info = timed_solve_env(env_increasing, save_q = True)
q_dictionary = info["q_dictionary"]

In [ ]:
len(q_dictionary.keys())

## Simulating trajectories

To construct trajectories, you need (1) a policy function (found in policies.py); (2) the inputs to that function (one of the functions from the last section); (3) an environment for the simulated agent to be acting on. For example, to simulate a trajectory with a softmax policy:

In [ ]:
from mouselab.policies import SoftmaxPolicy
from mouselab.agents import Agent

agent = Agent()

pol = SoftmaxPolicy(q_dictionary)
agent.register(pol)

agent.register(env_increasing)

trace = agent.run_episode()

In [ ]:
trace

The output will be a dictionary containing information from the simulated episode, including states, actions and rewards. (Check out agents.py for more information and a function for running many episodes.)